In [1]:
import nltk

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
 
dl = nltk.downloader.Downloader('http://nltk.org/nltk_data/index.xml')
# nltk.download('stopwords')
dl.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words('english')    
#print(stopwords) # see default stopwords

def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

[nltk_data] Error loading stopwords: <urlopen error [Errno 60]
[nltk_data]     Operation timed out>


In [2]:
# Params for bert model and tokenization
Nsamp = 200 # number of samples to generate in each class - 'spam', 'not spam'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token
Enron_Sample = 30000

In [3]:
def reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', "", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

In [4]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        try:
            tokens = row.split(" ")[:maxtokens]
        except:
            tokens=""
    return tokens

In [5]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

#jose data

In [6]:
import os
jose_phishing_files = []

for dirname, _, filenames in os.walk('../data/jose_phishing_html'):
    for filename in filenames:
        if filename.startswith('phishing'):
            jose_phishing_files.append(os.path.join(dirname, filename))
jose_phishing_files[0]    

'../data/jose_phishing_html/phishing5.mbox-1805.eml.html'

In [7]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data
    
jose_file_contents = get_data_content(jose_phishing_files)
len(jose_file_contents)

10147

In [8]:
import pandas as pd
jose_bodies_df = pd.DataFrame(jose_file_contents,columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

jose_bodies_df.head() # yo

,message
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
1,"<html dir=""ltr"">\n<head>\n<meta http-equiv=""Co..."
2,"<TABLE cellSpacing=0 cellPadding=0 width=""100%..."
3,Your mailbox quota has been exceeded the stora...
4,"<style type=""text/css"">\n.n {\n\tcolor: #FF000..."


In [9]:
len(jose_bodies_df)

10147

#Enron email data.

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
filepath = "../data/enron/emails.csv"

# Read the enron data into a pandas.DataFrame called emails
emails = pd.read_csv(filepath)

print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))
emails.head()

Successfully loaded 517401 rows and 2 columns!


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [11]:
#emails.loc[100]["message"]


In [12]:
import email

def extract_messages(df):
    messages = []
    for item in df["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages

bodies = extract_messages(emails)

Successfully retrieved message body from e-mails!


In [13]:
len(bodies)

517401

In [14]:
import random
bodies_df = pd.DataFrame(random.sample(bodies, Enron_Sample))

# expand default pandas display options to make emails more clearly visible when printed
pd.set_option('display.max_colwidth', 300)

bodies_df.head() # 

,0
0,"Enron Methanol nominates the following natural gas requirements for the \nMethanol Plant for September 2000:\n\n34,000 mmbtu per day\n\nEGPFC nominates the following natural gas requirements for the MTBE Plant at \nMorgan's Point for September 2000:\n\n9,000 mmbtu per day"
1,"Dear Tim,\n\nThank you for your feedback.\n\nThe weekly meetings are going well. We're actually meeting every other week and I am providing DPR impact reports and feedback to the traders on a weekly basis. I find that meeting every other Wednesday gives me a chance to monitor trends per each d..."
2,"Please send me a copy of the pleading. fax # 713-646-3393\n\n\nTo: Richard B Sanders/HOU/ECT@ECT\ncc: \nSubject: Washington County, OH purchases\n\nI wanted to make sure you are aware of this since it may be an ECT issue. \nThis is a very minor case ($60K) and the only reason Enron is a named ..."
3,"This # is not working???\n\n\n\n\ncjackson@txuenergy.com on 10/13/2000 10:50:56 AM\nTo: mike.carson@enron.com\ncc: \n\nSubject: \n\n\n\n\nCarson,\n\nCall me when you get a chance...214 975-9806...I don't have your office\nnumber...\n\nJackson\n\n\n\n"
4,"FYI - \n\n -----Original Message-----\nFrom: Jafry, Rahil \nSent: Monday, March 26, 2001 11:14 AM\nTo: Beck, Sally; Hall, Bob\nCc: Abbanat, Stephen\nSubject: FW: CMS Opportunity\n\nFYI - Lets talk about this today.\n\n -----Original Message-----\nFrom: gregory.felton@cgiusa.com@ENRON \n[mailt..."


#fraudulent corpus

In [15]:
filepath = "../data/fradulent_emails.txt"
with open(filepath, 'r',encoding="latin1") as file:
    data = file.read()
    
# split on a code word appearing close to the beginning of each email
fraud_emails = data.split("From r")
len(fraud_emails)

3978

In [16]:
#fraud_emails[1]


In [17]:
fraud_bodies = extract_messages(pd.DataFrame(fraud_emails, columns=["message"],dtype=str))
fraud_bodies_df = pd.DataFrame(fraud_bodies[1:])

fraud_bodies_df.head() # yo

Successfully retrieved message body from e-mails!


,0
0,"FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY G..."
1,"Dear Friend,\n\nI am Mr. Ben Suleman a custom officer and work as Assistant controller of the Customs and Excise department Of the Federal Ministry of Internal Affairs stationed at the Murtala Mohammed International Airport, Ikeja, Lagos-Nigeria.\n\nAfter the sudden death of the former Head of s..."
2,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
3,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
4,"Dear sir, \n \nIt is with a heart full of hope that I write to seek your help in respect of the context below. I am Mrs. Maryam Abacha the former first lady of the former Military Head of State of Nigeria General Sani Abacha whose sudden death occurred on 8th of June 1998 as a result of cardiac ..."


#IWSPA


In [18]:
# paths of the input files
no_head_train_path_0 = '../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../data/phishing/IWSPA-AP-traindata/legit/'
head_train_path_0 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Phish/'
head_train_path_1 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Legit/'
no_head_test_path = '../data/phishing/IWSPA-APTestData/testdata_noheaders/'
head_test_path = '../data/phishing/IWSPA-APTestData/testdata_fullheaders/'

In [19]:
import os, re, string
import numpy as np

In [20]:
def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [21]:

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        text_list.append(cleaned_text)
    return text_list, files

In [22]:
no_head_train_phish, temp = get_data(no_head_train_path_0)
no_head_train_legit, temp = get_data(no_head_train_path_1)
#head_train_0, temp = get_data(head_train_path_0)
#head_train_1, temp = get_data(head_train_path_1)
no_head_test, no_head_files = get_data(no_head_test_path)
#head_test, head_files = get_data(head_test_path)

In [23]:
# no_head_train = no_head_train_phish + no_head_train_legit
# no_head_labels_train = ([0] * len(no_head_train_phish)) + ([1] * len(no_head_train_legit))

#head_train = head_train_0 + head_train_1
#head_labels_train = ([0] * len(head_train_0)) + ([1] * len(head_train_1))

In [24]:
import pandas as pd
aw_legit_df = pd.DataFrame(no_head_train_legit, columns=["message"],dtype=str)
aw_phish_df = pd.DataFrame(no_head_train_phish, columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [25]:
aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [26]:
# legit
EnronEmails = bodies_df.iloc[:,0]#.apply(tokenize)
# EnronEmails = bodies_df.iloc[:,0].apply(tokenize)
# EnronEmails = EnronEmails.apply(stop_word_removal)
# EnronEmails = EnronEmails.apply(reg_expressions)
# EnronEmails = EnronEmails.sample(len(EnronEmails))


# iwspa
IwspaEmails_legit =  aw_legit_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_legit = IwspaEmails_legit.sample(len(IwspaEmails_legit))

# phishing
fraudEmails = fraud_bodies_df.iloc[:,0]#.apply(tokenize)
# fraudEmails = fraud_bodies_df.iloc[:,0].apply(tokenize)
# fraudEmails = fraudEmails.apply(stop_word_removal)
# fraudEmails = fraudEmails.apply(reg_expressions)
# fraudEmails = fraudEmails.sample(len(fraudEmails))

# iwspa
IwspaEmails_phish =  aw_phish_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_phish = IwspaEmails_phish.sample(len(IwspaEmails_phish))
# jose
JoseEmails = jose_bodies_df.iloc[:, 0]#.apply(tokenize)
# JoseEmails = JoseEmails.sample(len(JoseEmails))


raw_data = pd.concat([fraudEmails, IwspaEmails_phish, JoseEmails, EnronEmails, IwspaEmails_legit], axis=0).values


In [27]:
len(raw_data)

49845

In [28]:
# 1:spam 0:not spam
print("Shape of combined data represented as numpy array is:")
print(raw_data.shape)
#print("Data represented as numpy array is:")
#print(raw_data)

# corresponding labels
Categories = ['spam','notspam']
#header = ([1]*Nsamp)
#header.extend(([0]*Nsamp))
header = ([1]*(len(fraudEmails) + len(IwspaEmails_phish) + len(JoseEmails)))
header.extend(([0]*(len(EnronEmails) + len(IwspaEmails_legit))))

Shape of combined data represented as numpy array is:
(49845,)


In [29]:
type(raw_data[10])

str

In [30]:
# function for shuffling data in unison with labels/header
def unison_shuffle(a, b):
    p = np.random.permutation(len(b))
    data = a[p]
    header = np.asarray(b)[p]
    return data, header

# function for converting data into the right format, due to the difference in required format from sklearn models
# we expect a single string per email here, versus a list of tokens for the sklearn models previously explored
def convert_data(raw_data,header):
    converted_data, labels = [], []
    for i in range(raw_data.shape[0]):
        # out = ' '.join(str(raw_data[i]))
        out = str(raw_data[i])
        converted_data.append(out)
        labels.append(header[i])
        #print(i)
    converted_data = np.array(converted_data, dtype=object)[:, np.newaxis]
    
    return converted_data, np.array(labels)

raw_data, header = unison_shuffle(raw_data, header)

# split into independent 70% training and 30% testing sets
idx = int(0.7*raw_data.shape[0])
# 70% of data for training
train_x, train_y = convert_data(raw_data[:idx],header[:idx])
# remaining 30% for testing
test_x, test_y = convert_data(raw_data[idx:],header[idx:])

print("train_x/train_y list details, to make sure it is of the right form:")
# print(len(train_x))
# print(train_x)
# print(train_y[:5])
print(train_y.shape)
print(test_y.shape)


train_x/train_y list details, to make sure it is of the right form:
34891
(34891,)


In [31]:
type(train_x)


numpy.ndarray

In [32]:
np.save('../output/train_x.npy', train_x)
np.save('../output/train_y.npy', train_y)
np.save('../output/test_x.npy', test_x)
np.save('../output/test_y.npy', test_y)

In [33]:
b = np.load('../output/train_x.npy', allow_pickle=True)

issue = np.array_equal(train_x,b)  #验证  比较a，b是否相同时同一个array
print(str(issue))

b = np.load('../output/train_y.npy', allow_pickle=True)

issue = np.array_equal(train_y,b)
print(str(issue))

b = np.load('../output/test_x.npy', allow_pickle=True)

issue = np.array_equal(test_x,b)  #验证  比较a，b是否相同时同一个array
print(str(issue))

b = np.load('../output/test_y.npy', allow_pickle=True)

issue = np.array_equal(test_y,b)
print(str(issue))

True
True
True
True


In [34]:
store = pd.concat([pd.DataFrame(train_x), pd.DataFrame(train_y)], axis=1).values
store1 = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y)], axis=1).values

In [35]:
store[0]


array(['Looks good to me.  Are we certain that all of the other traders plan to post \ndaily volumes?\n\n\n\n\nDavid Forster\n10/08/99 09:51 AM\nTo: Elena Kapralova/LON/ECT@ECT, Mark E Taylor/HOU/ECT@ECT\ncc: Bob Shults/HOU/ECT@ECT, Marcello Romano/LON/ECT@ECT, Mark \nDilworth/LON/ECT@ECT \nSubject: Re: Paper Volumes  \n\nThat looks fine to me. Please ensure the Volume tab is consistent with this \ndescription as well.\n\nMark: Please comment if you have any concerns.\n\nThanks,\n\nDave\n\n\n\nElena Kapralova\n08/10/99 12:18\nTo: David Forster/LON/ECT@ECT\ncc: Bob Shults/HOU/ECT@ECT, Marcello Romano/LON/ECT@ECT, Mark \nDilworth/LON/ECT@ECT \nSubject: Re: Paper Volumes  \n\nthe current paper descriptions says:\n\n"....Seller pays a Floating Price and the Buyer pays the price submitted by \nCounterparty on the website (the Fixed Price) in each case in respect of the \nNotional Quantity per Determination Period.  Each calendar month during the \nterm of the Transaction will be a Determina

In [36]:
csv = pd.DataFrame(store)
csv.to_csv('../output/train_1.csv')

In [37]:
csv = pd.DataFrame(store1)
csv.to_csv('../output/train_2.csv')

#enron html data

In [38]:
# def extract_html_messages(df):
#     messages = []
#     for item in df["message"]:
#         # Return a message object structure from a string
#         e = email.message_from_string(item)
#         # get message body
#         message_body = e.get_payload()
#         if '<html>' in message_body:
#             messages.append(message_body)
#         # if BeautifulSoup(message_body, "html.parser").find():
#         #     messages.append(message_body)
#     print("Successfully retrieved message body from e-mails!")
#     print(len(messages))
#     return messages
# enron_html_bodies = extract_html_messages(emails)

In [39]:
# enron_html_bodies[0]


#positive: legit
#enron text:50W html:874
#iwspa text:5091 html:9

#negative: phishing
#fraudulent text:3978 html:147
#iwspa text:628 html:15
#jose text:10147 html:10147



#dataset:
#fradulent all 
#jose all html
#iwspa all no header
#enron 2W text + all html